### Introduction
This notebook is designed to run inference on the [Diffuser](https://arxiv.org/abs/2205.09991) planning model for model-based RL. The notebook is modified from the authors' [original](https://colab.research.google.com/drive/1YajKhu-CUIGBJeQPehjVPJcK_b38a8Nc?usp=sharing#scrollTo=57hSzI4mCgat). For those new to reinforcement learning, consider checking out the HuggingFace [Reinforcement Learning Course](https://huggingface.co/blog/deep-rl-intro) for a primer.

> Colab made by [Nathan Lambert](https://natolambert.com) and [Ben Glickenhaus](https://www.linkedin.com/in/benjamin-glickenhaus-859532a3).

![diffusers_library](https://github.com/huggingface/diffusers/raw/main/docs/source/imgs/diffusers_library.jpg)


### Installing Packages

#### `apt-get install` requirements 

These requirements primarily pertain to install mujoco and run it in the colab.
Source was inspired by this (fairly recent) [demo](https://colab.research.google.com/drive/1KGMZdRq6AemfcNscKjgpRzXqfhUtCf-V?usp=sharing).

In [ ]:
# installations primiarly needed for Mujoco
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglew-dev is already the newest version (2.0.0-5).
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-glx is already the newest version (20.0.8-0ubuntu1~18.04.1).
libosmesa6-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
patchelf is already the newest version (0.9-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


#### Install Diffusers

In [ ]:
%cd /content

# install latest HF diffusers
!rm -rf /content/diffusers/
!git clone -b rl https://github.com/huggingface/diffusers.git
!pip install -q /content/diffusers 
!pip install -q datasets transformers 

/content
Found existing installation: diffusers 0.5.0.dev0
Uninstalling diffusers-0.5.0.dev0:
  Successfully uninstalled diffusers-0.5.0.dev0
Cloning into 'diffusers'...
remote: Enumerating objects: 10356, done.
remote: Counting objects: 100% (502/502), done.
remote: Compressing objects: 100% (251/251), done.
remote: Total 10356 (delta 277), reused 384 (delta 201), pack-reused 9854
Receiving objects: 100% (10356/10356), 7.81 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (6885/6885), done.
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel 

#### `pip install` requirements

In [ ]:
# primarily RL-sepcific requirements
%pip install -f https://download.pytorch.org/whl/torch_stable.html \
                free-mujoco-py \
                einops \
                gym==0.24.1 \
                protobuf==3.20.1 \
                git+https://github.com/rail-berkeley/d4rl.git \
                mediapy \
                Pillow==9.0.0 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Cloning https://github.com/rail-berkeley/d4rl.git to /tmp/pip-req-build-7j2y8u6t
  Running command git clone -q https://github.com/rail-berkeley/d4rl.git /tmp/pip-req-build-7j2y8u6t
  Cloning https://github.com/aravindr93/mjrl (to revision master) to /tmp/pip-install-g98wzheg/mjrl_0abe064c9aa541e98742a70535434798
  Running command git clone -q https://github.com/aravindr93/mjrl /tmp/pip-install-g98wzheg/mjrl_0abe064c9aa541e98742a70535434798


#### Import D4RL to initialize Mujoco
[Mujoco](https://github.com/deepmind/mujoco) is a physics simulator used extensively in reinforcement learning research. Here, we import [D4RL](https://github.com/rail-berkeley/d4rl) (a library of datasets and environments for Offline RL), which results in the building of Mujoco.

In [ ]:
## cythonize mujoco-py at first import
import d4rl

No module named 'flow'
No module named 'carla'
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:416: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  "The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead."




---



### Environment & Model Setup
In this section, we will create the environment, handle the data, and run the diffusion model.

#### Imports



In [ ]:
import torch
import tqdm
import numpy as np
import gym 

#### Create environment
This colab is designed to run with pretrained models from the hopper environment. As more models are trained, this can be extended.


In [ ]:
env_name = "hopper-medium-v2"
env = gym.make(env_name)
data = env.get_dataset() # dataset is only used for normalization in this colab

/usr/local/lib/python3.7/dist-packages/gym/envs/mujoco/mujoco_env.py:47: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  "This version of the mujoco environments depends "
/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:70: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  "Agent's minimum action space value is -infinity. This is probably too low."
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:74: UserWarning: WARN: Agent's m

#### Define constants

In [ ]:
# Cuda settings for colab
torch.cuda.get_device_name(0)
DEVICE = 'cuda:0'
DTYPE = torch.float

# diffusion model settings
n_samples = 4   # number of trajectories planned via diffusion
horizon = 128   # length of sampled trajectories
state_dim = env.observation_space.shape[0] 
action_dim = env.action_space.shape[0]
num_inference_steps = 20 # number of difusion steps

#### Helper functions
* `normalize` scales the state values corresponding to the training data-set in D4RL,
* `de_normalize` unscales the data for correct rendering,
* `to_torch` handles casting to torch for both numpy arrays and dicts (used for conditionning the model, see `reset_x0`).

In [ ]:
def normalize(x_in, data, key):
    means = data[key].mean(axis=0)
    stds = data[key].std(axis=0)
    return (x_in - means) / stds


def de_normalize(x_in, data, key):
    means = data[key].mean(axis=0)
    stds = data[key].std(axis=0)
    return x_in * stds + means
	
def to_torch(x_in, dtype=None, device=None):
	dtype = dtype or DTYPE
	device = device or DEVICE
	if type(x_in) is dict:
		return {k: to_torch(v, dtype, device) for k, v in x_in.items()}
	elif torch.is_tensor(x_in):
		return x_in.to(device).type(dtype)
	return torch.tensor(x_in, dtype=dtype, device=device)


#### Sample env. initial state

In [ ]:
## Can set environment seed for debugging
# torch.manual_seed(0)
# np.random.seed(0)
# env.seed(1996)

obs = env.reset()
obs_raw = obs

# normalize observations for forward passes
obs = normalize(obs, data, 'observations')

### Run the Diffusion Process -- from Scratch

#### Initialize model
In this section, we create a scheduler and load a pretrained model from the Hub. An important detail in the RL application space is to save `conditions` which will allow the model to optimize trajectories only from the current state (which is cruical to making decisions!). 

In [ ]:
from diffusers import DDPMScheduler, UNet1DModel

# Two generators for different parts of the diffusion loop to work in colab
generator = torch.Generator(device='cuda')
generator_cpu = torch.Generator(device='cpu')

scheduler = DDPMScheduler(num_train_timesteps=100,beta_schedule="squaredcos_cap_v2")

# The horizion represents the length of trajectories used in training.
network = UNet1DModel.from_pretrained("bglick13/hopper-medium-v2-value-function-hor32", subfolder="unet").to(device=DEVICE)

#### Planning helper function
`reset_x0` is used to constrain the diffusion process to trajectories starting at the current state of the agent. 
Without this, the diffusion process would generate arbitrary high-reward trajectories, rather than trajectories beginning at the current state.

In [ ]:
def reset_x0(x_in, cond, act_dim):
	for key, val in cond.items():
		x_in[:, key, act_dim:] = val.clone()
	return x_in

#### Setup for denoising
`conditions` is the variable used to hold the first state of the planned trajectories to the current state (it is passed into `reset_x0`).

In [ ]:
## add a batch dimension and repeat for multiple samples
## [ observation_dim ] --> [ n_samples x observation_dim ]
obs = obs[None].repeat(n_samples, axis=0)
conditions = {
    0: to_torch(obs, device=DEVICE)
  }

# constants for inference
batch_size = len(conditions[0])
shape = (batch_size, horizon, state_dim+action_dim)

#### Sample initial noise

In [ ]:
# sample random initial noise vector
x1 = torch.randn(shape, device=DEVICE, generator=generator)

# this model is conditioned from an initial state, so you will see this function
#  multiple times to change the initial state of generated data to the state 
#  generated via env.reset() above or env.step() below
x = reset_x0(x1, conditions, action_dim)

# convert a np observation to torch for model forward pass
x = to_torch(x)


#### Generate trajectories
The diffusion process for trajectories has 4 central components:
1. sampling an predicted original sample from the model (note that this model directly predicts the sample, rather than the error term `epsilon` used in many diffusion models),
2. use the scheduler to predict the sample at the previous timestep,
3. [optional] add posterior noise to the sample,
4. condition the trajectory to constrain the initial state.

In [ ]:
eta = 1.0 # noise factor for sampling reconstructed state

# run the diffusion process
# for i in tqdm.tqdm(reversed(range(num_inference_steps)), total=num_inference_steps):
for i in tqdm.tqdm(scheduler.timesteps):

    # create batch of timesteps to pass into model
    timesteps = torch.full((batch_size,), i, device=DEVICE, dtype=torch.long)
    
    # 1. generate prediction from model
    with torch.no_grad():
      residual = network(x.permute(0, 2, 1), timesteps).sample
      residual = residual.permute(0, 2, 1) # needed to match model params to original 

    # 2. use the model prediction to reconstruct an observation (de-noise)
    obs_reconstruct = scheduler.step(residual, i, x, predict_epsilon=False)["prev_sample"]

    # 3. [optional] add posterior noise to the sample
    if eta > 0:
      noise = torch.randn(obs_reconstruct.shape, generator=generator_cpu).to(obs_reconstruct.device)
      posterior_variance = scheduler._get_variance(i) # * noise
      # no noise when t == 0
      # NOTE: original implementation missing sqrt on posterior_variance
      obs_reconstruct = obs_reconstruct + int(i>0) * (0.5 * posterior_variance) * eta* noise  # MJ had as log var, exponentiated

    # 4. apply conditions to the trajectory
    obs_reconstruct_postcond = reset_x0(obs_reconstruct, conditions, action_dim)
    x = to_torch(obs_reconstruct_postcond)


100%|██████████| 100/100 [00:01<00:00, 78.56it/s]


In [ ]:
x.shape

torch.Size([4, 128, 14])



---



### Render the samples

#### Renderering Tools
Rendering from Mujoco is historically not easy. Here is a modified version from the original paper. Additionally, a TODO is to investigate this web-based [viewer](https://github.com/kevinzakka/mjc_viewer).

##### Video helpers

In [ ]:
import os
import mediapy as media

def to_np(x_in):
	if torch.is_tensor(x_in):
		x_in = x_in.detach().cpu().numpy()
	return x_in

# from MJ's Diffuser code 
# https://github.com/jannerm/diffuser/blob/76ae49ae85ba1c833bf78438faffdc63b8b4d55d/diffuser/utils/colab.py#L79
def mkdir(savepath):
    """
        returns `True` iff `savepath` is created
    """
    if not os.path.exists(savepath):
        os.makedirs(savepath)
        return True
    else:
        return False


def show_sample(renderer, observations, filename='sample.mp4', savebase='/content/videos'):
    '''
    observations : [ batch_size x horizon x observation_dim ]
    '''

    mkdir(savebase)
    savepath = os.path.join(savebase, filename)

    images = []
    for rollout in observations:
        ## [ horizon x height x width x channels ]
        img = renderer._renders(rollout, partial=True)
        images.append(img)

    ## [ horizon x height x (batch_size * width) x channels ]
    images = np.concatenate(images, axis=2)

    media.show_video(images, codec='h264', fps=60)

##### Renderer helpers
These functions involve setting the state of the environment and reading it out in a pixel form.

In [ ]:
# Code adapted from Michael Janner
# source: https://github.com/jannerm/diffuser/blob/main/diffuser/utils/rendering.py
import mujoco_py as mjc

def env_map(env_name):
    '''
        map D4RL dataset names to custom fully-observed
        variants for rendering
    '''
    if 'halfcheetah' in env_name:
        return 'HalfCheetahFullObs-v2'
    elif 'hopper' in env_name:
        return 'HopperFullObs-v2'
    elif 'walker2d' in env_name:
        return 'Walker2dFullObs-v2'
    else:
        return env_name

def get_image_mask(img):
    background = (img == 255).all(axis=-1, keepdims=True)
    mask = ~background.repeat(3, axis=-1)
    return mask

def atmost_2d(x):
    while x.ndim > 2:
        x = x.squeeze(0)
    return x

def set_state(env, state):
    qpos_dim = env.sim.data.qpos.size
    qvel_dim = env.sim.data.qvel.size
    if not state.size == qpos_dim + qvel_dim:
        warnings.warn(
            f'[ utils/rendering ] Expected state of size {qpos_dim + qvel_dim}, '
            f'but got state of size {state.size}')
        state = state[:qpos_dim + qvel_dim]

    env.set_state(state[:qpos_dim], state[qpos_dim:])


##### Rendering class
Use the previously defined helpers to programatically render pixel sequences from a trajectory of states. 
This class takes the re-scaled outputs of the diffusion process and visualizes them.

In [ ]:
class MuJoCoRenderer:
    '''
        default mujoco renderer
    '''

    def __init__(self, env):
        if type(env) is str:
            env = env_map(env)
            self.env = gym.make(env)
        else:
            self.env = env
        ## - 1 because the envs in renderer are fully-observed
        ## @TODO : clean up
        self.observation_dim = np.prod(self.env.observation_space.shape) - 1
        self.action_dim = np.prod(self.env.action_space.shape)
        try:
            self.viewer = mjc.MjRenderContextOffscreen(self.env.sim)
        except:
            print('[ utils/rendering ] Warning: could not initialize offscreen renderer')
            self.viewer = None

    def pad_observation(self, observation):
        state = np.concatenate([
            np.zeros(1),
            observation,
        ])
        return state

    def pad_observations(self, observations):
        qpos_dim = self.env.sim.data.qpos.size
        ## xpos is hidden
        xvel_dim = qpos_dim - 1
        xvel = observations[:, xvel_dim]
        xpos = np.cumsum(xvel) * self.env.dt
        states = np.concatenate([
            xpos[:,None],
            observations,
        ], axis=-1)
        return states

    def render(self, observation, dim=256, partial=False, qvel=True, render_kwargs=None, conditions=None):

        if type(dim) == int:
            dim = (dim, dim)

        if self.viewer is None:
            return np.zeros((*dim, 3), np.uint8)

        if render_kwargs is None:
            xpos = observation[0] if not partial else 0
            render_kwargs = {
                'trackbodyid': 2,
                'distance': 3,
                'lookat': [xpos, -0.5, 1],
                'elevation': -20
            }

        for key, val in render_kwargs.items():
            if key == 'lookat':
                self.viewer.cam.lookat[:] = val[:]
            else:
                setattr(self.viewer.cam, key, val)

        if partial:
            state = self.pad_observation(observation)
        else:
            state = observation

        qpos_dim = self.env.sim.data.qpos.size
        if not qvel or state.shape[-1] == qpos_dim:
            qvel_dim = self.env.sim.data.qvel.size
            state = np.concatenate([state, np.zeros(qvel_dim)])

        set_state(self.env, state)

        self.viewer.render(*dim)
        data = self.viewer.read_pixels(*dim, depth=False)
        data = data[::-1, :, :]
        return data

    def _renders(self, observations, **kwargs):
        images = []
        for observation in observations:
            img = self.render(observation, **kwargs)
            images.append(img)
        return np.stack(images, axis=0)

    def renders(self, samples, partial=False, **kwargs):
        if partial:
            samples = self.pad_observations(samples)
            partial = False

        sample_images = self._renders(samples, partial=partial, **kwargs)

        composite = np.ones_like(sample_images[0]) * 255

        for img in sample_images:
            mask = get_image_mask(img)
            composite[mask] = img[mask]

        return composite

    def __call__(self, *args, **kwargs):
        return self.renders(*args, **kwargs)

#### Show Plans
This section renders 4 trajectories chosen from the same initial state in the environment.

##### Initialize renderer class for the environment

In [ ]:
render = MuJoCoRenderer(env)

##### Show the video
Show the states generated by the diffusion model in the real environment. 
Not that the actions are dropped from the data.

In [ ]:
de_normalized = de_normalize(to_np(x[:,:,action_dim:]), data, 'observations')
show_sample(render, de_normalized)


This browser does not support the video tag.


# Run Value Guided Diffusion -- with Pipeline

In this section, we repeat the above code, but we use a pre-trained pipeline in Diffusers!

In [ ]:
from diffusers import ValueGuidedRLPipeline

In [ ]:
env_name = "hopper-medium-v2"
env = gym.make(env_name)
data = env.get_dataset()  # dataset is only used for normalization in this colab
render = MuJoCoRenderer(env)

/usr/local/lib/python3.7/dist-packages/gym/envs/mujoco/mujoco_env.py:47: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  "This version of the mujoco environments depends "
/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:112: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:70: UserWarning: WARN: Agent's minimum action space value is -infinity. This is probably too low.
  "Agent's minimum action space value is -infinity. This is probably too low."
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:74: UserWarning: WARN: Agent's m

In [ ]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
DEVICE = "cuda"

Load the pipeline!

In [ ]:
pipeline = ValueGuidedRLPipeline.from_pretrained(
        "bglick13/hopper-medium-v2-value-function-hor32",
        env=env,
    )

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

The config attributes {'args': ['diffusers', 'DDPMScheduler'], 'kwargs': ['diffusers', 'PNDMScheduler']} were passed to ValueGuidedDiffuserPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
load datafile: 100%|██████████| 21/21 [00:01<00:00, 13.65it/s]


In [ ]:
env.seed(0)
obs = env.reset()
total_reward = 0
total_score = 0
T = 100
rollout = [obs.copy()]
trajectories = []
y_maxes = [0]
for t in tqdm.tqdm(range(T)):
    # normalize observations for forward passes
    denorm_actions = pipeline(obs, planning_horizon=32)

    # execute action in environment
    next_observation, reward, terminal, _ = env.step(denorm_actions)
    score = env.get_normalized_score(total_reward)
    
    # update return
    total_reward += reward
    total_score += score
    print(
        f"Step: {t}, Reward: {reward}, Total Reward: {total_reward}, Score: {score}, Total Score:"
        f" {total_score}"
    )
    # save observations for rendering
    rollout.append(next_observation.copy())

    obs = next_observation


/usr/local/lib/python3.7/dist-packages/gym/core.py:201: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  "Function `env.seed(seed)` is marked as deprecated and will be removed in the future. "
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:217: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator. 
  "Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator. "
/usr/local/lib/python3.7/dist-packages/gym/utils/passive_env_checker.py:229: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  "Future gym versions will require that `Env.reset` can b

Step: 0, Reward: 0.9613453417633487, Total Reward: 0.9613453417633487, Score: 0.006228869141685884, Total Score: 0.006228869141685884



  2%|▏         | 2/100 [00:03<02:38,  1.62s/it]

Step: 1, Reward: 0.9905865761281842, Total Reward: 1.951931917891533, Score: 0.006524252144941468, Total Score: 0.012753121286627353



  3%|▎         | 3/100 [00:04<02:25,  1.50s/it]

...

Step: 97, Reward: 1.6721521590810136, Total Reward: 126.77269361223948, Score: 0.04466726587386679, Total Score: 2.607237057668249



 99%|█████████▉| 99/100 [02:20<00:01,  1.62s/it]

Step: 98, Reward: 1.75613883059031, Total Reward: 128.52883244282978, Score: 0.04518105140461443, Total Score: 2.652418109072863



100%|██████████| 100/100 [02:22<00:00,  1.42s/it]

Step: 99, Reward: 1.7563279535050194, Total Reward: 130.2851603963348, Score: 0.045720642682980664, Total Score: 2.698138751755844


In [ ]:
show_sample(render, np.expand_dims(np.stack(rollout), axis=0))

This browser does not support the video tag.
